### Day 6: Guard Gallivant

Link: https://adventofcode.com/2024/day/6#part2

We can solve part two of the problem by simulating the guard's walk through the map, as before, while also simulating adding an obstacle to each potential new step that:

1. Does not leave the map.
2. Does not run into an obstacle.
3. Has not been visited before, given the obstacle should be added at the beginning, not while the guard is walking. This also excludes the initial position where the guard starts, following the problem's restrictions.

If adding the obstacle creates a loop, we increase the result. To detect a loop during the simulation, we include the guard's direction in the visited states. Revisiting the same place with the same direction indicates a loop has been found.

This solution has a time complexity of `O(n²)`, which is manageable given the input size of 130 x 130 and the restriction against simulating obstacles in visited places.

In [ ]:
# Please ensure there is an `input.txt` file in this folder containing your input.
lines: list[str] = []


with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
guard_map: list[list[str]] = []
guard_start_row, guard_start_column = -1, -1


for row, line in enumerate(lines):
    map_row: list[str] = []

    for column, char in enumerate(line.strip()):
        map_row.append(char)

        if char == "^":
            guard_start_row, guard_start_column = row, column

    guard_map.append(map_row)


loop_count = 0
guard_row, guard_column = guard_start_row, guard_start_column
actions = [
    (-1, 0),  # Up
    (0, 1),  # Right
    (1, 0),  # Down
    (0, -1),  # Left
]
direction = 0  # Up
visited = {(guard_row, guard_column)}


def has_left_map(row: int, column: int) -> bool:
    return not(0 <= row < len(guard_map)) or not(0 <= column < len(guard_map[0]))


def is_obstacle(guard_map: list[list[str]], row: int, column: int) -> bool:
    return guard_map[row][column] == "#"


def turn_right(direction: int) -> int:
    return (direction + 1) % len(actions)


def does_obstacle_create_loop(
    guard_row: int,
    guard_column: int,
    obstacle_row: int,
    obstacle_column: int,
    direction: int,
) -> bool:
    visited: set[tuple[int, int, int]] = set()

    while True:
        new_row, new_column = guard_row + actions[direction][0], guard_column + actions[direction][1]

        if has_left_map(new_row, new_column):
            break

        is_placed_obstacle = (new_row, new_column) == (obstacle_row, obstacle_column)

        if is_placed_obstacle or is_obstacle(guard_map, new_row, new_column):
            direction = turn_right(direction)

            if (guard_row, guard_column, direction) in visited:  # Loop detected
                return True

            visited.add((guard_row, guard_column, direction))
            continue

        if (new_row, new_column, direction) in visited:  # Loop detected
            return True

        guard_row, guard_column = new_row, new_column
        visited.add((guard_row, guard_column, direction))

    return False


while True:
    new_row, new_column = guard_row + actions[direction][0], guard_column + actions[direction][1]

    if has_left_map(new_row, new_column):
        break

    if is_obstacle(guard_map, new_row, new_column):
        direction = turn_right(direction)
        continue

    has_visited = (new_row, new_column) in visited

    if not has_visited and does_obstacle_create_loop(
        guard_row, guard_column, new_row, new_column, direction
    ):
        loop_count += 1

    guard_row, guard_column = new_row, new_column
    visited.add((guard_row, guard_column))


print(loop_count)